<a href="https://colab.research.google.com/github/Khaled-Mohammed-Abdelgaber/deep-learning-projects-/blob/main/transfer_learning_for_cat_dog_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [7]:
import os
import shutil

# Creating required directory

In [5]:
original_dataset_dir = '/content/drive/MyDrive/Colab Notebooks/train' # where all datasets will be downloaded
base_dir = '/content/cats_and_dogs_small'     # where small datasets will be stored
os.mkdir(base_dir)

#where train datasets(part of small datasets will be stored)
train_dir = os.path.join(base_dir, 'train')
print(f"Train directory is {train_dir}")
os.mkdir(train_dir)

#where validation datasets will be stored
validation_dir = os.path.join(base_dir,'validation')
os.mkdir(validation_dir)

#where test datasets will be stored
test_dir = os.path.join(base_dir,'test')
os.mkdir(test_dir)

#where cats train datasets will be stored
train_cats_dir = os.path.join(train_dir,'cats')
os.mkdir(train_cats_dir)

#where dogs train datasets will be stored
train_dogs_dir = os.path.join(train_dir,'dogs')
os.mkdir(train_dogs_dir)

#where cats validation datasets will be stored
validation_cats_dir = os.path.join(validation_dir, 'cats')
os.mkdir(validation_cats_dir)

#where dogs validation datasets will be stored
validation_dogs_dir = os.path.join(validation_dir, 'dogs')
os.mkdir(validation_dogs_dir)

#where cats test datasets will be stored
test_cats_dir = os.path.join(test_dir, 'cats')
os.mkdir(test_cats_dir)

#where dogs test datasets will be stored
test_dogs_dir = os.path.join(test_dir, 'dogs')
os.mkdir(test_dogs_dir)


Train directory is /content/cats_and_dogs_small/train


# Moving datasets to required paths

In [8]:
fnames = ['dog.{}.jpg'.format(i)for i in range(1000)]
for fname in fnames:
  src = os.path.join(original_dataset_dir,fname)
  dst = os.path.join(train_dogs_dir,fname)
  shutil.copyfile(src , dst)


fnames = ['dog.{}.jpg'.format(i)for i in range(1000,1500)]
for fname in fnames:
  src = os.path.join(original_dataset_dir,fname)
  dst = os.path.join(validation_dogs_dir,fname)
  shutil.copyfile(src , dst)


fnames = ['dog.{}.jpg'.format(i)for i in range(1500,2000)]
for fname in fnames:
  src = os.path.join(original_dataset_dir,fname)
  dst = os.path.join(test_dogs_dir,fname)
  shutil.copyfile(src , dst)



In [9]:
fnames = ['cat.{}.jpg'.format(i)for i in range(1000)]
for fname in fnames:
  src = os.path.join(original_dataset_dir,fname)
  dst = os.path.join(train_cats_dir,fname)
  shutil.copyfile(src , dst)

fnames = ['cat.{}.jpg'.format(i)for i in range(1000,1500)]
for fname in fnames:
  src = os.path.join(original_dataset_dir,fname)
  dst = os.path.join(validation_cats_dir,fname)
  shutil.copyfile(src , dst)

fnames = ['cat.{}.jpg'.format(i)for i in range(1500,2000)]
for fname in fnames:
  src = os.path.join(original_dataset_dir,fname)
  dst = os.path.join(test_cats_dir,fname)
  shutil.copyfile(src , dst)

#Loading VGG16 network

In [10]:
from keras.applications.vgg16 import VGG16
conv_base = VGG16(weights = 'imagenet',
                  include_top = False,
                  input_shape = (150,150,3))

conv_base.summary()

58900480/58889256 [==============================] - 0s 0us/step
Model: "vgg16"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 150, 150, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 150, 150, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 150, 150, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 75, 75, 64)        0         
                                                                 
 block2_conv1 (Conv2D)       (None, 75, 75, 128)       73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 75, 75, 128)       147584    
                                                              

# Freezing up to block 4

In [14]:
conv_base.trainable = True
set_trainable = False
for layer in conv_base.layers:
  if layer.name == 'block5_conv1':
    set_trainable = True
  if set_trainable:
    layer.trainable = True
  else:
    layer.trainable = False

# Stacking VGG16 with the classifier

In [21]:
from keras import models
from keras import layers
from tensorflow.keras import optimizers
model = models.Sequential()
model.add(conv_base)
model.add(layers.Flatten())
model.add(layers.Dense(256, activation='relu'))
model.add(layers.Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy',
            optimizer=optimizers.RMSprop(lr=1e-5),
            metrics=['acc'])

In [22]:
import os
import shutil
import numpy as np
from keras.preprocessing.image import ImageDataGenerator
#base_dir = '/content/drive/MyDrive/Colab Notebooks'     # where small datasets will be stored
#train_dir = os.path.join(base_dir, 'train')
#validation_dir = os.path.join(base_dir, 'validation')
#test_dir = os.path.join(base_dir, 'test')

# Data Augmentation

In [23]:
datagen = ImageDataGenerator(rescale=1./255)
batch_size = 20
def extract_features(directory , sample_count):
  features = np.zeros(shape = (sample_count , 4,4,512))
  labels = np.zeros(shape=(sample_count))
  generator = datagen.flow_from_directory(directory,
                                            target_size=(150, 150),
                                            batch_size=batch_size,
                                            class_mode='binary')
  i=0
  for inputs_batch , labels_batch in generator :
    features_batch = conv_base.predict(inputs_batch)
    features[i*batch_size : (i+1)*batch_size] = features_batch
    labels[i*batch_size : (i+1)*batch_size] = labels_batch
    i += 1
    if i * batch_size >= sample_count:
      break
  return features, labels

                                            

In [24]:
from keras.preprocessing.image import ImageDataGenerator
from keras import optimizers
train_datagen = ImageDataGenerator(
rescale=1./255,
rotation_range=40,
width_shift_range=0.2,
height_shift_range=0.2,
shear_range=0.2,
zoom_range=0.2,
horizontal_flip=True,
fill_mode='nearest')

In [25]:
test_datagen = ImageDataGenerator(rescale=1./255)
train_generator = train_datagen.flow_from_directory(
train_dir,
target_size=(150, 150),
batch_size=20,
class_mode='binary')
validation_generator = test_datagen.flow_from_directory(
validation_dir,
target_size=(150, 150),
batch_size=20,
class_mode='binary')

Found 2000 images belonging to 2 classes.
Found 1000 images belonging to 2 classes.


In [26]:
from tensorflow.keras import optimizers
model.compile(loss='binary_crossentropy',
                optimizer=optimizers.RMSprop(lr=2e-5),
                metrics=['acc']
              )
history = model.fit_generator(train_generator,
                              steps_per_epoch=100,
                              epochs=10,
                              validation_data=validation_generator,
                              validation_steps=50)

Epoch 1/10


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:10: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  # Remove the CWD from sys.path while we load stuff.


100/100 [==============================] - 21s 199ms/step - loss: 0.2397 - acc: 0.8920 - val_loss: 0.1962 - val_acc: 0.9190
Epoch 2/10
100/100 [==============================] - 20s 198ms/step - loss: 0.1680 - acc: 0.9325 - val_loss: 0.3066 - val_acc: 0.8910
Epoch 3/10
100/100 [==============================] - 20s 197ms/step - loss: 0.1492 - acc: 0.9420 - val_loss: 0.1565 - val_acc: 0.9340
Epoch 4/10
100/100 [==============================] - 20s 197ms/step - loss: 0.1400 - acc: 0.9425 - val_loss: 0.1689 - val_acc: 0.9320
Epoch 5/10
100/100 [==============================] - 20s 196ms/step - loss: 0.1228 - acc: 0.9475 - val_loss: 0.2023 - val_acc: 0.9270
Epoch 6/10
100/100 [==============================] - 20s 197ms/step - loss: 0.1187 - acc: 0.9505 - val_loss: 0.3025 - val_acc: 0.9120
Epoch 7/10
100/100 [==============================] - 20s 197ms/step - loss: 0.1095 - acc: 0.9590 - val_loss: 0.1726 - val_acc: 0.9340
Epoch 8/10
100/100 [==============================] - 20s 196ms/st

In [27]:
model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 vgg16 (Functional)          (None, 4, 4, 512)         14714688  
                                                                 
 flatten_2 (Flatten)         (None, 8192)              0         
                                                                 
 dense_4 (Dense)             (None, 256)               2097408   
                                                                 
 dense_5 (Dense)             (None, 1)                 257       
                                                                 
Total params: 16,812,353
Trainable params: 9,177,089
Non-trainable params: 7,635,264
_________________________________________________________________


In [28]:
test_generator = test_datagen.flow_from_directory(test_dir,
                                                  target_size=(150, 150),
                                                  batch_size=20,
                                                  class_mode='binary')
                                                      
test_loss, test_acc = model.evaluate_generator(test_generator, steps=50)
print('test acc:', test_acc)

Found 1000 images belonging to 2 classes.


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:6: UserWarning: `Model.evaluate_generator` is deprecated and will be removed in a future version. Please use `Model.evaluate`, which supports generators.
  


test acc: 0.9409999847412109
